# Projeto 1 - Ciência dos Dados

Nome: João Pedro Reis Lima

Nome: Lucas Gurgel 

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [66]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [67]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Joao\Projeto-1-CDADOS


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [68]:
filename = 'esquadrão suicida 2.xlsx'

In [69]:
dados = pd.read_excel(filename)
dados.head(5)

,Treinamento,relevancia
0,to feliz q da p prestar atenção e gostar de vá...,1.0
1,os roteirista de esquadrão suicida 2 devem ter...,0.0
2,assistir esse esquadrão suicida 2 de marola,0.0
3,e eu e gilson que fechamos a sala de cinema p ...,0.0
4,"esquadrão suicida 2 é muito bom, amei o filme",1.0


In [70]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,relevancia
0,assisti esquadrão suicida 2 com um pouco de at...,0
1,"vou ver esquadrão suicida 2 hj, vcs já viram?",0
2,@lsdcomixxx adm hj eu sonhei que ele tava no e...,0
3,ranking personagens do esquadrão suicida:\n\n1...,1
4,mano gloria groove no esquadrão suicida 2 fico...,1


___
## Classificador automático de sentimento


Nós consideramos relavante comentários que demostravam alguma critica ao produto tais como: Se a pessoa gostou ou não da obra, o que ela sentiu(emoção), também consideramos personagens favoritos como algo relavante 

Não consideramos comentarios de situações adversas sem relação direta com a criticidade da obra, tal como comentarios pessoais

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [71]:
import re 
import emoji
from emoji import UNICODE_EMOJI


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[”@\-/!.:?;,''"|()#$%¨&*]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [72]:
def minusculo(text):
    return text.lower()

In [73]:
def separa_emoji(tweet):
   
    modified=' '.join(emoji.get_emoji_regexp().split(tweet))
    modified=modified.split()
    for i,emoji1 in enumerate(modified):
        if emoji1 in UNICODE_EMOJI['pt']:
            modified[i]=UNICODE_EMOJI['pt'][emoji1].replace(':','')
        else:
            continue
    modified=' '.join(modified)
        
    return modified

In [74]:
def limpatudo(text):
    tira_pontuacao = cleanup(text)
    tudo_minusculo = minusculo(tira_pontuacao)
    limpo = (separa_emoji(tudo_minusculo))
    
    return limpo



In [75]:
def lista(df):
    listaf = []
    '''
    p = ''
    
    for i in df:
        p+=i + ' '
        '''
    lista = df.values.tolist() 
    for i in lista:
        for palavra in i.split():
            listaf.append(palavra)
        
    return listaf

In [76]:
#Transformando palavras em variáveis categóricas:
dados['Treinamento'] = dados['Treinamento'].astype('category')
test['Teste'] = test['Teste'].astype('category')

In [77]:
dados['Treinamento']=dados['Treinamento'].apply(limpatudo)
test['Teste']=test['Teste'].apply(limpatudo)

In [78]:
ir = dados['relevancia'] == 0
r = dados['relevancia'] == 1

dados_r = dados.loc[r,:]
dados_ir = dados.loc[ir,:]

dados_r

,Treinamento,relevancia
0,to feliz q da p prestar atenção e gostar de vá...,1.0
4,esquadrão suicida 2 é muito bom amei o filme,1.0
5,henriquenarizz kkkkkk amigo mas pior que o esq...,1.0
8,ruanfalco dccomics tô p assistir esquadrão sui...,1.0
16,ei gostei de esquadrão suicida 2 e é só minha ...,1.0
...,...,...
290,achei esquadrão suicida 2 chatokkkk,1.0
291,fui assistir o esquadrão suicida 2 muito gore ...,1.0
294,nossa depende hq's dc e tvcinema a marvel poré...,1.0
296,vi esquadrão suicida 2 hj com minhas irmã e é ...,1.0


In [79]:
ir = test['relevancia'] == 0
r = test['relevancia'] == 1

test_r = test.loc[r,:]
test_ir = test.loc[ir,:]

test_ir

,Teste,relevancia
0,assisti esquadrão suicida 2 com um pouco de at...,0
1,vou ver esquadrão suicida 2 hj vcs já viram,0
2,lsdcomixxx adm hj eu sonhei que ele tava no es...,0
5,homemaranha em o esquadrão suicida 2 confirmad...,0
6,queria jogat lol mas quero ver esquadrão suici...,0
...,...,...
189,atenção acabei de assistir esquadrão suicida 2...,0
190,aquela cena em esquadrão suicida 2 que a harle...,0
191,esquadrão suicida 2 parece um filme de zack sn...,0
192,esquadrão suicida 2 vai entrar na hbo mds eu o...,0


In [80]:
pdr = lista(dados_r.Treinamento)

pdir = lista(dados_ir.Treinamento)

ptr = lista(test_r.Teste)

ptir = lista(test_ir.Teste)



ptrc = pd.Series(ptr) 
ptirc = pd.Series(ptir) 
pdrc = pd.Series(pdr) 
pdirc = pd.Series(pdir) 



In [81]:
fpdr=pdrc.value_counts()
fpdir=pdirc.value_counts()
fptr=ptrc.value_counts()
fptir=ptirc.value_counts()



In [82]:
listatr = ptrc.tolist()
listatir = ptirc.tolist()
listadr = pdrc.tolist()
listadir = pdirc.tolist()
lista_r = listatr + listadr
lista_ir = listatir + listadir
lista_total2 = pd.Series(lista_r + lista_ir)

lista_total1 = pd.Series(listadr+listadir)

Banco_de_dados=set(listadr+listadir)


lista_total1.value_counts()


for i in Banco_de_dados:
    if i == 'joao':
        print('HETAR')

In [83]:
# probabilidade de ser relevante:
pr = len(listadr)/len(lista_total1)
# probabilidade de ser irrelevante:
pi = len(listadir)/len(lista_total1)



print(pr+pi)



1.0


In [90]:
def laplace(freqabsoluta,conjunto):
    """""
    Retorna a probabilidade independente da palavra analisada pertencer ou não
    a determinada classe
    """
    return (freqabsoluta+1)/((conjunto)+len(Banco_de_dados))

In [91]:
def prob_R(text):
    Prob1 = 1
    for i in text.split():
        if i in fpdr:

            Alaplace_relevante = laplace(fpdr[i], pr)
        
            Prob1 *= Alaplace_relevante
        else:
            Alaplace_relevante = laplace(0, pr)
        
            Prob1 *= Alaplace_relevante
    return Prob1

print(prob_R('assistir esse esquadrão suicida 2 de marola'))



3.476983646985585e-13


In [92]:
def prob_I(text):
    Prob2 = 1
    for i in text.split():
        if i in fpdir: # FDP

            Alaplace_irrelevante = laplace(fpdir[i], pi)
        
            Prob2 *= Alaplace_irrelevante
        else:
            Alaplace_irrelevante = laplace(0, pi)
        
            Prob2 *= Alaplace_irrelevante
    return Prob2


In [93]:
def probabilidades(text):
    if prob_R(text)*pr > prob_I(text)*pi:
        return 1
    else:
        return 0

print(probabilidades('vi esquadrão suicida 2 hj com minhas irmã ')) 

0


In [88]:
dados['Classificado'] = dados.Treinamento.apply(probabilidades)

In [89]:
dados

,Treinamento,relevancia,Classificado
0,to feliz q da p prestar atenção e gostar de vá...,1.0,1
1,os roteirista de esquadrão suicida 2 devem ter...,0.0,0
2,assistir esse esquadrão suicida 2 de marola,0.0,0
3,e eu e gilson que fechamos a sala de cinema p ...,0.0,0
4,esquadrão suicida 2 é muito bom amei o filme,1.0,1
...,...,...,...
295,eu sou humano e posso sonhar homem de aço 2 fo...,0.0,0
296,vi esquadrão suicida 2 hj com minhas irmã e é ...,1.0,1
297,1 eu ainda não vi o esquadrão suicida então eu...,0.0,0
298,harrymetalfan tem esquadrao suicida 2 pensei q...,0.0,0


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**